In [3]:
setwd("~/Upd-Germline-Genomics")
source("_targets.R")

tar_load(repli.peaks_chr)
tar_load(repli.static.timing.peaks)

Loading required package: viridisLite


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [7]:
names(repli.peaks_chr)

[1] "Germline_Somatic"     "Germline_Kc167"       "Germline_S2"         
[4] "Somatic_Kc167"        "Somatic_S2"           "Kc167_S2"            
[7] "Dynamic_Static_Model"

In [4]:
tar_read(chromosome_pericetromere_label)

Loading required package: GenomicRanges

Loading required package: stats4

Loading required package: BiocGenerics




Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors


Attaching package: ‘S4Vectors’


The following objects are masked from ‘package:dplyr’:

    first, rename


The following object is masked from ‘package:utils’:

    findMatches


The following objects are masked from ‘package:base’:

    expand.grid, I, unname


Loading required pac

GRanges object with 4 ranges and 0 metadata columns:
      seqnames            ranges strand
         <Rle>         <IRanges>  <Rle>
  [1]       2L 22192401-23513712      *
  [2]       2R         1-5651400      *
  [3]       3L 23154101-28110227      *
  [4]       3R         1-4229200      *
  -------
  seqinfo: 4 sequences from an unspecified genome; no seqlengths

In [5]:
regions <- setNames(
  GRanges(
    c(
      "2L:1-22192400",
      "2L:22192401-23513712",
      "2R:1-5651400",
      "2R:5651401-25286936",
      "3L:1-23154100",
      "3L:23154101-28110227",
      "3R:1-4229200",
      "3R:4229201-32079331",
      "4:1-1348131",
      "X:1-23542271",
      "Y:1-3667352"
    )
  ),
  c(
    "2L",
    "2LC",
    "2RC",
    "2R",
    "3L",
    "3LC",
    "3RC",
    "3R",
    "4",
    "X",
    "Y"
  )
) %>%
  print()

GRanges object with 11 ranges and 0 metadata columns:
      seqnames            ranges strand
         <Rle>         <IRanges>  <Rle>
   2L       2L        1-22192400      *
  2LC       2L 22192401-23513712      *
  2RC       2R         1-5651400      *
   2R       2R  5651401-25286936      *
   3L       3L        1-23154100      *
  3LC       3L 23154101-28110227      *
  3RC       3R         1-4229200      *
   3R       3R  4229201-32079331      *
    4        4         1-1348131      *
    X        X        1-23542271      *
    Y        Y         1-3667352      *
  -------
  seqinfo: 7 sequences from an unspecified genome; no seqlengths


In [26]:
report <- cross_join(
  tibble(
    name = c(
      "Germline > Somatic",
      "Somatic > Germline",
      "Germline > Kc167",
      "Kc167 > Germline",
      "Germline > S2",
      "S2 > Germline",
      "Somatic > Kc167",
      "Kc167 > Somatic",
      "Somatic > S2",
      "S2 > Somatic",
      "Kc167 > S2",
      "S2 > Kc167",
      "4 Cell Types Static"
    ) %>%
      factor(., .),
    ranges = list(
      repli.peaks_chr[[1]] %>% subset(grepl("Earlier", names(.))),
      repli.peaks_chr[[1]] %>% subset(grepl("Later", names(.))),
      repli.peaks_chr[[2]] %>% subset(grepl("Earlier", names(.))),
      repli.peaks_chr[[2]] %>% subset(grepl("Later", names(.))),
      repli.peaks_chr[[3]] %>% subset(grepl("Earlier", names(.))),
      repli.peaks_chr[[3]] %>% subset(grepl("Later", names(.))),
      repli.peaks_chr[[4]] %>% subset(grepl("Earlier", names(.))),
      repli.peaks_chr[[4]] %>% subset(grepl("Later", names(.))),
      repli.peaks_chr[[5]] %>% subset(grepl("Earlier", names(.))),
      repli.peaks_chr[[5]] %>% subset(grepl("Later", names(.))),
      repli.peaks_chr[[6]] %>% subset(grepl("Earlier", names(.))),
      repli.peaks_chr[[6]] %>% subset(grepl("Later", names(.))),
      repli.static.timing.peaks
    )
  ),
  tibble(region = names(regions) %>% factor(., .))
) %>%
  group_by(name, region) %>%
  reframe(
    tribble(
      ~feature, ~value,
      "count",
      length(to(findOverlaps(regions[region[[1]]], ranges[[1]]))),
      "length_kb",
      ranges[[1]][
        to(findOverlaps(regions[region[[1]]], ranges[[1]]))
      ] %>%
        restrict(start(regions[region[[1]]]), end(regions[region[[1]]])) %>%
        width() %>%
        sum() %>%
        `/`(1000) %>%
        round(),
      "pct_covered",
      ranges[[1]][
        to(findOverlaps(regions[region[[1]]], ranges[[1]]))
      ] %>%
        restrict(start(regions[region[[1]]]), end(regions[region[[1]]])) %>%
        width() %>%
        sum() %>%
        `/`(width(regions[region[[1]]])) %>%
        round(digits = 3),
    )
  ) %>%
  print()

# A tibble: 429 × 4
   name               region feature        value
   <fct>              <fct>  <chr>          <dbl>
 1 Germline > Somatic 2L     count         60    
 2 Germline > Somatic 2L     length_kb   5060    
 3 Germline > Somatic 2L     pct_covered    0.228
 4 Germline > Somatic 2LC    count          8    
 5 Germline > Somatic 2LC    length_kb    772    
 6 Germline > Somatic 2LC    pct_covered    0.584
 7 Germline > Somatic 2RC    count         20    
 8 Germline > Somatic 2RC    length_kb   1447    
 9 Germline > Somatic 2RC    pct_covered    0.256
10 Germline > Somatic 2R     count         42    
# ℹ 419 more rows


In [27]:
data <- report %>%
  tibble(column = interaction(feature, name)) %>%
  acast(region ~ column, value.var = "value") %>%
  print()

    count.Germline > Somatic length_kb.Germline > Somatic
2L                        60                         5060
2LC                        8                          772
2RC                       20                         1447
2R                        42                         3124
3L                        54                         4260
3LC                       12                         1313
3RC                        5                          206
3R                        86                         6407
4                          4                          258
X                         40                         2222
Y                          1                           51
    pct_covered.Germline > Somatic count.Somatic > Germline
2L                           0.228                       42
2LC                          0.584                        0
2RC                          0.256                        1
2R                           0.159                       31
3L  

In [28]:
write.table(data, row.names = T, col.names = F, quote = F, sep = "\t")

2L	60	5060	0.228	42	3267	0.147	65	6384	0.288	41	4551	0.205	68	4264	0.192	56	4679	0.211	45	4188	0.189	49	3282	0.148	45	3323	0.15	50	4560	0.205	54	4743	0.214	64	6665	0.3	66	6101	0.275
2LC	8	772	0.584	0	0	0	7	384	0.291	4	148	0.112	6	358	0.271	3	376	0.284	0	0	0	6	560	0.424	1	41	0.031	2	59	0.044	5	486	0.368	1	82	0.062	1	87	0.066
2RC	20	1447	0.256	1	39	0.007	28	1581	0.28	14	756	0.134	16	891	0.158	14	723	0.128	0	0	0	9	1739	0.308	2	83	0.015	16	1112	0.197	14	1553	0.275	10	1146	0.203	1	47	0.008
2R	42	3124	0.159	31	2518	0.128	50	4195	0.214	50	3991	0.203	58	4264	0.217	37	3502	0.178	33	2426	0.124	60	3786	0.193	46	2268	0.116	41	2879	0.147	81	5434	0.277	51	3745	0.191	70	7335	0.374
3L	54	4260	0.184	43	3806	0.164	70	7058	0.305	50	4615	0.199	72	4477	0.193	53	4012	0.173	51	4193	0.181	46	3365	0.145	56	4263	0.184	56	4082	0.176	69	6221	0.269	69	5698	0.246	76	7073	0.305
3LC	12	1313	0.265	0	0	0	27	1768	0.357	7	829	0.167	13	686	0.138	12	669	0.135	0	0	0	12	871	0.176	0	0	0	13	719	0.145	4	213	0.043	19	1605	0.324	